In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
 #Import libraries
import os  # For interacting with the file system
import shutil  # For managing files and directories in a cross-platform manner
import keras  # For building deep learning models
import numpy as np  # For numerical operations on arrays
from glob import glob  # For finding file paths
from tqdm import tqdm  # For progress bars

# Data preprocessing
from keras.preprocessing.image import ImageDataGenerator  # For image data augmentation

# Data visualization
import seaborn as sns  # For statistical visualizations
import plotly.graph_objs as go  # For interactive visualizations
import matplotlib.pyplot as plt  # For creating static plots

# Model architecture
from keras import Sequential  # For building sequential models
from keras.models import load_model  # For loading pre-trained models
from keras.layers import Dense, GlobalAvgPool2D as GAP, Dropout  # For defining model layers

# Training callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping  # For training callbacks

# Pre-trained models
from tensorflow.keras.applications import InceptionV3, Xception, ResNet152V2  # For using pre-trained models

In [ ]:
# Set the path to the dataset
data_path = '/content/drive/MyDrive/animals'

# Get a list of class names from the data path
class_names = sorted(os.listdir(data_path))

# Count the number of classes
num_classes = len(class_names)

# Print the class names and the total number of classes
print("Class Names: \n", class_names)
print("Number of Classes:", num_classes)

Class Names: 
 ['antelope', 'badger', 'bat', 'bear', 'bee', 'beetle', 'bison', 'boar', 'butterfly', 'cat', 'caterpillar', 'chimpanzee', 'cockroach', 'cow', 'coyote', 'crab', 'crow', 'deer', 'dog', 'dolphin', 'donkey', 'dragonfly', 'duck', 'eagle', 'elephant', 'flamingo', 'fly', 'fox', 'goat', 'goldfish', 'goose', 'gorilla', 'grasshopper', 'hamster', 'hare', 'hedgehog', 'hippopotamus', 'hornbill', 'horse', 'hummingbird', 'hyena', 'jellyfish', 'kangaroo', 'koala', 'ladybugs', 'leopard', 'lion', 'lizard', 'lobster', 'mosquito', 'moth', 'mouse', 'octopus', 'okapi', 'orangutan', 'otter', 'owl', 'ox', 'oyster', 'panda', 'parrot', 'pelecaniformes', 'penguin', 'pig', 'pigeon', 'porcupine', 'possum', 'raccoon', 'rat', 'reindeer', 'rhinoceros', 'sandpiper', 'seahorse', 'seal', 'shark', 'sheep', 'snake', 'sparrow', 'squid', 'squirrel', 'starfish', 'swan', 'tiger', 'turkey', 'turtle', 'whale', 'wolf', 'wombat', 'woodpecker', 'zebra']
Number of Classes: 90


In [ ]:
# Get the number of samples in each class
class_sizes = []
for name in class_names:
    class_size = len(os.listdir(data_path + "/" + name))
    class_sizes.append(class_size)

# Print the class distribution
print("Class Distribution:\n", class_sizes)

Class Distribution:
 [60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 70, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60]


In [ ]:
# to convert lists to dictionary **************
class_name_size = dict(zip(class_names, class_sizes))

In [ ]:
class_name_size

{'antelope': 60,
 'badger': 60,
 'bat': 60,
 'bear': 60,
 'bee': 60,
 'beetle': 60,
 'bison': 60,
 'boar': 60,
 'butterfly': 60,
 'cat': 60,
 'caterpillar': 60,
 'chimpanzee': 60,
 'cockroach': 60,
 'cow': 60,
 'coyote': 60,
 'crab': 60,
 'crow': 60,
 'deer': 60,
 'dog': 60,
 'dolphin': 60,
 'donkey': 60,
 'dragonfly': 60,
 'duck': 60,
 'eagle': 60,
 'elephant': 60,
 'flamingo': 60,
 'fly': 60,
 'fox': 60,
 'goat': 60,
 'goldfish': 60,
 'goose': 60,
 'gorilla': 60,
 'grasshopper': 60,
 'hamster': 60,
 'hare': 60,
 'hedgehog': 60,
 'hippopotamus': 60,
 'hornbill': 60,
 'horse': 60,
 'hummingbird': 60,
 'hyena': 60,
 'jellyfish': 60,
 'kangaroo': 60,
 'koala': 60,
 'ladybugs': 60,
 'leopard': 60,
 'lion': 60,
 'lizard': 60,
 'lobster': 60,
 'mosquito': 60,
 'moth': 60,
 'mouse': 60,
 'octopus': 60,
 'okapi': 60,
 'orangutan': 60,
 'otter': 60,
 'owl': 60,
 'ox': 60,
 'oyster': 60,
 'panda': 60,
 'parrot': 60,
 'pelecaniformes': 60,
 'penguin': 60,
 'pig': 60,
 'pigeon': 60,
 'porcupine':

In [ ]:
# Initialize Generator with the specified image transformations and preprocessing
# rescale: normalizes pixel values from 0-255 to 0-1
# horizontal_flip: randomly flips images horizontally
# vertical_flip: randomly flips images vertically
# rotation_range: randomly rotates images by a given range in degrees
# validation_split: splits the data into training and validation sets, with 20% of the data used for validation
data_generator = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=20,
    validation_split=0.2)

# Load training data from the specified directory and apply the generator
# target_size: resizes the images to a specified size
# class_mode: specifies the type of label encoding, binary for 2 classes
# batch_size: specifies the number of samples per batch
# shuffle: shuffles the data after each epoch
# subset: specifies the subset of data to load, in this case, the training set
train_data = data_generator.flow_from_directory(
    data_path,
    target_size=(256,256),
    class_mode='binary',
    batch_size=32,
    shuffle=True,
    subset='training')

# Load validation data from the specified directory and apply the generator
# subset: specifies the subset of data to load, in this case, the validation set
valid_data = data_generator.flow_from_directory(
    data_path,
    target_size=(256,256),
    class_mode='binary',
    batch_size=32,
    shuffle=True,
    subset='validation')

Found 4328 images belonging to 90 classes.
Found 1082 images belonging to 90 classes.


In [ ]:
def show_image(image, image_title=None):
    '''
    This function takes in an image and an optional title and plots the image.
    '''
    # Display the image
    plt.imshow(image)

    # Set the title of the plot if provided
    plt.title(image_title)

    # Turn off the axes in the plot
    plt.axis('off')

In [ ]:
def get_random_data(data_tuple):
    """
    Function to get a random data point from a given dataset.

    Args:
    data_tuple (tuple): A tuple containing the dataset images and labels as numpy arrays.

    Returns:
    A random image and its corresponding label as numpy arrays.
    """
    images, labels = data_tuple
    # get a random index for an image in the dataset
    idx = np.random.randint(len(images))

    # select the image and its corresponding label using the random index
    image, label = images[idx], labels[idx]

    # return the selected image and label
    return image, label

In [ ]:
    #return image, label
# Set the figure size for the plot
plt.figure(figsize=(20,20))

# Initialize a counter for the subplots
counter=1

# Loop over the train dataset
for images, labels in iter(train_data):

      # Get a random image and label
      image, label = get_random_data([images, labels])

      # Plot the image with its class name as the title
      plt.subplot(5,5,counter)
      show_image(image, image_title=f"Class : {class_names[int(label)]}")

      # Increment the counter
      counter+=1

      # End the loop when 25 images have been plotted
      if counter>=26: break

# Adjust the layout and display the plot
plt.tight_layout()
plt.show()

In [ ]:
name = "ResNet152V2"

# Load the pre-trained ResNet152V2 model, freeze its weights and exclude its final classification layer.
base_model = ResNet152V2(include_top=False, input_shape=(256,256,3), weights='imagenet')
base_model.trainable = False

# Create a sequential model with the ResNet152V2 base model, a global average pooling layer, two fully connected layers, and a final softmax classification layer.
resnet152V2 = Sequential([
    base_model,
    GAP(),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(num_classes, activation='softmax')
], name=name)

# Compile the model with sparse categorical cross-entropy as the loss function, Adam optimizer and accuracy as the evaluation metric.
resnet152V2.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# Set up the EarlyStopping and ModelCheckpoint callbacks to monitor the training process and save the best model weights.
cbs = [
    EarlyStopping(patience=3, restore_best_weights=True),
    ModelCheckpoint(name + ".h5", save_best_only=True)
]

# Train the model using the training and validation datasets, using 50 epochs and the previously defined callbacks.
resnet152V2.fit(
    train_data, validation_data=valid_data,
    epochs=20, callbacks=cbs
)

Epoch 1/20
136/136 [==============================] - 144s 988ms/step - loss: 2.2967 - accuracy: 0.4778 - val_loss: 1.2077 - val_accuracy: 0.6867
Epoch 2/20
136/136 [==============================] - 131s 965ms/step - loss: 0.9903 - accuracy: 0.7364 - val_loss: 1.0148 - val_accuracy: 0.7292
Epoch 3/20
136/136 [==============================] - 132s 970ms/step - loss: 0.7383 - accuracy: 0.7918 - val_loss: 0.9876 - val_accuracy: 0.7357
Epoch 4/20
136/136 [==============================] - 132s 969ms/step - loss: 0.6295 - accuracy: 0.8149 - val_loss: 0.9147 - val_accuracy: 0.7514
Epoch 5/20
136/136 [==============================] - 132s 968ms/step - loss: 0.5293 - accuracy: 0.8445 - val_loss: 0.8381 - val_accuracy: 0.7736
Epoch 6/20
136/136 [==============================] - 132s 965ms/step - loss: 0.4608 - accuracy: 0.8602 - val_loss: 0.8204 - val_accuracy: 0.7800
Epoch 7/20
136/136 [==============================] - 131s 962ms/step - loss: 0.4057 - accuracy: 0.8794 - val_loss: 0.8538 -

In [ ]:
import cv2
import numpy as np

In [ ]:
model = load_model('/content/drive/MyDrive/ResNetakhil.h5')

In [ ]:
class_names = ['antelope', 'badger', 'bat', 'bear', 'bee', 'beetle', 'bison', 'boar', 'butterfly', 'cat', 'caterpillar', 'chimpanzee', 'cockroach', 'cow', 'coyote', 'crab', 'crow', 'deer', 'dog', 'dolphin', 'donkey', 'dragonfly', 'duck', 'eagle', 'elephant', 'flamingo', 'fly', 'fox', 'goat', 'goldfish', 'goose', 'gorilla', 'grasshopper', 'hamster', 'hare', 'hedgehog', 'hippopotamus', 'hornbill', 'horse', 'hummingbird', 'hyena', 'jellyfish', 'kangaroo', 'koala', 'ladybugs', 'leopard', 'lion', 'lizard', 'lobster', 'mosquito', 'moth', 'mouse', 'octopus', 'okapi', 'orangutan', 'otter', 'owl', 'ox', 'oyster', 'panda', 'parrot', 'pelecaniformes', 'penguin', 'pig', 'pigeon', 'porcupine', 'possum', 'raccoon', 'rat', 'reindeer', 'rhinoceros', 'sandpiper', 'seahorse', 'seal', 'shark', 'sheep', 'snake', 'sparrow', 'squid', 'squirrel', 'starfish', 'swan', 'tiger', 'turkey', 'turtle', 'whale', 'wolf', 'wombat', 'woodpecker', 'zebra']

def preprocess_image(image_path):
    # Load the image using OpenCV or Pillow
    img = cv2.imread(image_path)  # Example using OpenCV

    # Resize the image
    img = cv2.resize(img, (256, 256))  # Resize to 256x256 as in the training code

    # Convert the image to a NumPy array
    img = np.array(img)

    # Rescale the pixel values from 0-255 to 0-1
    img = img / 255.0

    return img





# Preprocess the test image
test_image = preprocess_image('/content/tigdeerjpg.jpg')

# Prepare the image for prediction
test_image = np.expand_dims(test_image, axis=0)

# Load the model (choose ResNet152V2 or Xception)
  # Replace 'my_image_classifier.h5' with your desired filename


# # Make predictions
# predictions = model.predict(test_image)

# # Get the predicted class index with the highest probability
# predicted_class_index = np.argmax(predictions[0])

# # Print the predicted class name
# print(f"Predicted Class: {class_names[predicted_class_index]}")


# # ... (Previous code, including preprocessing and loading the model)

# Make predictions
predictions = model.predict(test_image)
#print(predictions)
# Get the predicted class index with the highest probability
# Maximum probability among the predicted probabilities
max_probability = np.max(predictions)

# Threshold value (adjust as needed)
threshold = 0.5

# Check if the maximum probability is above the threshold
if max_probability >= threshold:
    # Get the predicted class index with the highest probability
    predicted_class_index = np.argmax(predictions[0])
    # Print the predicted class name
    print(f"Predicted Class: {class_names[predicted_class_index]}")
else:
    # Probability below threshold, classify as "unknown"
    print("Unknown Sample")




1/1 [==============================] - 1s 1s/step
Predicted Class: tiger (1.00)


In [ ]:
model.save('/content/drive/MyDrive/ResNetakhil.h5')